In [1]:
import nltk
import pandas as pd
import sys
import os

from data_loader import DataLoader

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.cluster.util import cosine_distance

import numpy

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import networkx as nx

import re

from collections import defaultdict
import string

ModuleNotFoundError: No module named 'spacy'

In [ ]:
loader = DataLoader(DataLoader.data_path2)
table = loader.load_table()
table.head()

In [ ]:
#Data Preprocess
#Get stopwords
nltk_stopwords = stopwords.words('english')
nltk_stopwords.append('\n')
spacy_stopwords = list(STOP_WORDS)
merged_sw = nltk_stopwords + list(set(spacy_stopwords) - set(nltk_stopwords))

print("nltk: ", len(nltk_stopwords))
print("spacy: ", len(spacy_stopwords))
print("merge: ", len(merged_sw))

In [ ]:
#Create the review text dictionary
reviewText = {}
for i in range(200):
    if table['asin'][i] in reviewText:
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)
    else:
        reviewText[table['asin'][i]] = []
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)
print(reviewText)

In [ ]:
def data_preprocess(original_review, stop_words):
    # to lowercase
    review = original_review.lower()
    # remove punctuation
    review = re.sub(r'[^\w\s]', ' ', review).strip()

    lemmatizer = WordNetLemmatizer()
    review_words = review.split(' ')
    review_words_tag = nltk.pos_tag(review_words)
    processed_review = ""
    # remove stop words&lemma
    for word, tag in review_words_tag:
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma_word = word
        else:
            lemma_word = lemmatizer.lemmatize(word, wntag)
        if lemma_word not in stop_words:
            processed_review = processed_review + lemma_word + " "
    processed_review = " ".join(processed_review.split())
    processed_review = processed_review.strip()
    return processed_review



In [ ]:
processed_reviewText = {}
for r in reviewText:
    processed_reviewText[r] = []
    for t in reviewText[r]:
        p = data_preprocess(t, merged_sw)
        processed_reviewText[r].append(p)
print(processed_reviewText)

In [ ]:
# collect frequency

def collect_frequency(processed_review):
    word_frequency = defaultdict(lambda:0)
    for v in processed_review.values():
        for i in v:
            tokens = nltk.word_tokenize(i)
            for token in tokens:
                word_frequency[token] += 1  
    return word_frequency

frequency = collect_frequency(processed_reviewText)

In [ ]:
frequency = sorted(frequency.items(), key=lambda item: item[1],reverse=True)
print(frequency)

In [ ]:
sorted_processed_reviewText = sorted(processed_reviewText.items(), key=lambda item: len(item[1]),reverse=True)
print(sorted_processed_reviewText)

In [ ]:
#B001ESSLIU 41> B004EI3ON4 34 > 9714721180 16
print(len(processed_reviewText['B001ESSLIU']))

In [ ]:
def build_similarity_matrix(sentencess, stopwords=None):
    similarity_matrix = numpy.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stopwords)
    return similarity_matrix

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

def generate_summary(sentences, top_n=5):
    summarize_text = []

    sentence_similarity_martix = build_similarity_matrix(sentences, None)

    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)
    for i in range(top_n):
          summarize_text.append(" ".join(ranked_sentence[i][1]))

    print("Summarize Text: \n", ". ".join(summarize_text))

In [ ]:
sentences = processed_reviewText['B001ESSLIU']
generate_summary(sentences, 2)

In [ ]:
# Abstractive Summarizer by Model
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

model_name = 'google/pegasus-xsum'
torch_device = 'cuda'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

text = """"""
for i in reviewText['9714721180']:
    text+=i
print(text)

batch = tokenizer.prepare_seq2seq_batch(text, truncation=True, padding='longest',return_tensors='pt').to(torch_device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

print(tgt_text)

In [ ]:
#rake_nltk to extract keyphrases
from rake_nltk import Rake

r = Rake()

r.extract_keywords_from_text(text)

r.get_ranked_phrases()[0:10]

#python-rake to extract keyphrases
import RAKE
rake_object = RAKE.Rake(merged_sw)
def sort_tuple(tup):
    tup.sort(key = lambda x:x[1])
    return tup
keywords = sort_tuple(rake_object.run(text))[-10:]
print(keywords)